In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import Sequence
import os
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from keras import layers
from keras import models
from keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score
from PIL import Image

In [14]:
# Define the root directory where your data is stored
root_directory = "dataset_2_final"

# Get the list of subdirectories (categories)
subdirectories = [subdir for subdir in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, subdir))]

# Create lists to store image paths and corresponding labels
image_paths = []
labels = []

# Iterate through each subdirectory (category)
for label, subdirectory in enumerate(subdirectories):
    # Construct the full path to the subdirectory
    subdirectory_path = os.path.join(root_directory, subdirectory)

    # Get a list of image files in the subdirectory
    image_files = [os.path.join(subdirectory_path, filename) for filename in os.listdir(subdirectory_path) if filename.endswith('.jpg')]  # Adjust the file extension as needed
    
    # Append image paths and labels
    image_paths.extend(image_files)
    labels.extend([label] * len(image_files))


# Now you have the image_paths and labels
print("Total images:", len(image_paths))
print("Total labels:", len(labels))

Total images: 494
Total labels: 494


In [15]:
# Define a custom data generator that inherits from the Sequence class
class CustomDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_samples = 4 * len(image_paths)
        self.indices = np.arange(self.num_samples)


    def __len__(self):
        
        return int(np.ceil(self.num_samples / self.batch_size))


    def __getitem__(self, index):

        # Determine the range of indices for the current batch
        batch_indices = self.indices[index * self.batch_size : (index + 1) * self.batch_size]


        batch_images = []
        batch_labels = []

        # Iterate over indices in the current batch
        for batch_index in batch_indices:

            # Calculate the index of the original image and the piece index within it
            image_index = batch_index // 4   # Divide by 4 to get original image index
            piece_index = batch_index % 4    # Modulus 4 to get piece index

            # Get the image path and original label for the current image
            image_path = self.image_paths[image_index]
            original_label = self.labels[image_index]
            image_pieces = self.load_image(image_path)

            # Load and split the image into pieces
            piece = image_pieces[piece_index]
            batch_images.append(piece)
            batch_labels.append(original_label)


        batch_images = np.array(batch_images)
        batch_labels = np.array(batch_labels)

        return batch_images, batch_labels

    def load_image(self, image_path):    

        # Load the original image using Pillow
        original_image = Image.open(image_path)
        original_image_array = np.array(original_image)

        # Split the original image into 4 pieces
        h, w, c = original_image_array.shape
        h_half, w_half = h // 2, w // 2
       
        image_pieces = [
            original_image_array[:h_half, :w_half],
            original_image_array[:h_half, w_half:],
            original_image_array[h_half:, :w_half],
            original_image_array[h_half:, w_half:]
        ]
        
        # Resize the image pieces and provide the shape of the output
        resized_images = [tf.image.resize(piece, self.image_size) for piece in image_pieces]

        #Shape of resized image list
        #print(np.array(resized_images).shape)    
        
        return resized_images


In [16]:

batch_size = 16
image_size = (380, 380)  # Adjust the image size based on your model's input requirements

# Split data into training, validation, and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.2, random_state=42)

# Create data generators for training, validation, and test sets
train_generator = CustomDataGenerator(train_paths, train_labels, batch_size, image_size)
val_generator = CustomDataGenerator(val_paths, val_labels, batch_size, image_size)
test_generator = CustomDataGenerator(test_paths, test_labels, batch_size, image_size)

In [17]:
def create_keras_model(hp):

    base_model = EfficientNetB4(include_top=False, weights='imagenet', pooling='avg')

    # Introduce a layer of data augmentation
    data_augmentation = Sequential([
        preprocessing.RandomRotation(0.2),
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomZoom(0.2),
        preprocessing.RandomContrast(0.2),
        preprocessing.RandomTranslation(0.2, 0.2),
        preprocessing.RandomHeight(0.2),
        preprocessing.RandomWidth(0.2),
    ])


    # Freeze all layers in the base model
    for layer in base_model.layers:
        layer.trainable = False
    # Unfreeze the last 10 layers in the base model for fine-tuning
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Capa de entradas
    entradas = layers.Input((380, 380, 3))
    # Capa de augmentation
    x = data_augmentation(entradas)
    # Pass the augmented images through the base model
    x = base_model(x)

    # TUNE the number of units in the dense layer
    hp_units = hp.Int('units', min_value=128, max_value=512, step=128)
    x = layers.Dense(units=hp_units, activation='relu')(x)
    # Add another dense layer
    salidas = layers.Dense(7, activation='softmax')(x)

    model = Model(inputs=entradas, outputs=salidas)
    
    #TUNE learning rate
    optimizer = keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model


In [18]:
from kerastuner.tuners import RandomSearch

In [19]:
# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    create_keras_model,  # Function to build the model
    objective='val_accuracy',  # Metric to optimize
    max_trials=15,  # Number of hyperparameter combinations to try
    directory='random_search',  # Directory to store results
    project_name='efficientenet_tuning'  # Name of the tuning project
)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 380, 380, 3)]     0         
_________________________________________________________________
sequential_2 (Sequential)    (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, 1792)              17673823  
_________________________________________________________________
dense_4 (Dense)              (None, 128)               229504    
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 903       
Total params: 17,904,230
Trainable params: 2,545,671
Non-trainable params: 15,358,559
_________________________________________________________________


In [20]:
# Search for the best hyperparameters
tuner.search(train_generator, epochs=15, validation_data=val_generator)

Trial 5 Complete [01h 23m 35s]
val_accuracy: 0.6645569801330566

Best val_accuracy So Far: 0.6645569801330566
Total elapsed time: 06h 56m 12s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
384               |256               |units
0.0064236         |0.00063324        |learning_rate

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 380, 380, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, 1792)              17673823  
_________________________________________________________________
dense (Dense)                (None, 384)               688512    
_________________________________________________________________
dense_

KeyboardInterrupt: 

In [21]:
# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
#Print a graph of the search
tuner.results_summary()


#Print all available information about the grid search
tuner.results_summary()


# Build and compile the best model
best_model = tuner.hypermodel.build(best_hyperparameters)

print("Best model summary: ")
best_model.summary()

Results summary
Results in random_search\efficientenet_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
units: 256
learning_rate: 0.0006332383741037616
Score: 0.6645569801330566

Trial 00 summary
Hyperparameters:
units: 512
learning_rate: 0.00040995319944661575
Score: 0.655063271522522

Trial 03 summary
Hyperparameters:
units: 384
learning_rate: 0.003956370567137552
Score: 0.6518987417221069

Trial 01 summary
Hyperparameters:
units: 384
learning_rate: 0.0009064135578205235
Score: 0.6329113841056824

Trial 02 summary
Hyperparameters:
units: 384
learning_rate: 0.002242865877928494
Score: 0.5981012582778931

Trial 05 summary
Hyperparameters:
units: 384
learning_rate: 0.006423597973604284
Results summary
Results in random_search\efficientenet_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
units: 256
learning_rate: 0.0006332383741037616
Score: 0.6645569801330566
